# Tutorial 3 venco.py

This tutorial aims to give a more in depth overview into the GridModeler class and showcases some features that can be customised.

In [4]:
from pathlib import Path
import matplotlib.pyplot as plt

from vencopy.core.dataparsers  import parse_data
from vencopy.core.flexestimators import FlexEstimator
from vencopy.core.gridmodellers import GridModeller
from vencopy.core.diarybuilders import DiaryBuilder
from vencopy.core.profileaggregators import ProfileAggregator
from vencopy.utils.utils import load_configs, create_output_folders

In [5]:
base_path = Path.cwd().parent / 'vencopy'
configs = load_configs(base_path)
create_output_folders(configs=configs)

# Adapt relative paths in config for tutorials
configs['dev_config']['global']['relative_path']['parse_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['parse_output']
configs['dev_config']['global']['relative_path']['diary_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['diary_output']
configs['dev_config']['global']['relative_path']['grid_output'] = Path.cwd().parent/ configs['dev_config']['global']['relative_path']['grid_output']
configs['dev_config']['global']['relative_path']['flex_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['flex_output']
configs['dev_config']['global']['relative_path']['aggregator_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['aggregator_output']
configs['dev_config']['global']['relative_path']['processor_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['processor_output']

# Set reference dataset
dataset_id = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configs["user_config"]["global"]["absolute_path"]["vencopy_root"] = Path.cwd()
configs['user_config']['global']['absolute_path'][dataset_id] = Path.cwd() /'data_sampling'

# Similarly we modify the dataset_id in the global config file
configs['dev_config']['global']['files'][dataset_id]['trips_data_raw'] = dataset_id + '.csv'


# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configs['dev_config']['dataparsers']['data_variables']['household_id']
del configs['dev_config']['dataparsers']['data_variables']['person_id']


## GridModeler config file

Let's print the GridModeler config file.

In [6]:
configs['user_config']['gridmodellers']

{'minimum_parking_time': 900,
 'grid_model': 'simple',
 'losses': True,
 'force_last_trip_home': True,
 'rated_power_simple': 11,
 'charging_infrastructure_mappings': {'DRIVING': False,
  'HOME': True,
  'WORK': True,
  'SCHOOL': False,
  'SHOPPING': True,
  'LEISURE': True,
  'OTHER': False,
  'NA': False,
  'PEOPLE_MOBILITY': False,
  'TRANSPORT': False,
  'SERVICES': False},
 'grid_availability_distribution': {'DRIVING': {0: 1},
  'HOME': {3.6: 0.2, 11: 0.2, 22: 0, 0: 0.6},
  'WORK': {11: 0.2, 22: 0.2, 0: 0.6},
  'SCHOOL': {11: 0, 22: 0, 0: 1},
  'SHOPPING': {11: 0.2, 22: 0.2, 0: 0.6},
  'LEISURE': {11: 0.2, 22: 0.2, 0: 0.6},
  'OTHER': {11: 0.2, 22: 0.2, 0: 0.6},
  '0.0': {0: 1},
  'PEOPLE_MOBILITY': {11: 0.2, 22: 0.2, 0: 0.6},
  'TRANSPORT': {11: 0.2, 22: 0.2, 0: 0.6},
  'SERVICES': {11: 0.2, 22: 0.2, 0: 0.6}},
 'loss_factor': {'rated_power_0': 0,
  'rated_power_3.6': 0.1,
  'rated_power_11': 0.1,
  'rated_power_22': 0.1,
  'rated_power_50': 0.2}}

As we can see the GridModeler config file contains two keys: chargingInfrastructureMappings and chargingInfrastructureDistributions. The first one basically sets for which trip purpose the infrastructure availability should be considered, the second one specifies the probabilities given for each location (trip purpose) and the respective charging power.

## _GridModeler_ class

The charging infrastructure allocation makes use of a basic charging infrastructure model, which assumes the availability of charging stations when vehicles are parked. Since the analytical focus of the framework lies on a regional level (NUTS1-NUTS0), the infrastructure model is kept simple in the current version.

Charging availability is allocated based on a binary True–False mapping to a respective trip purpose in the venco.py config. Thus, different scenarios describing different charging availability scenarios, e.g., at home or at home and at work etc. can be distinguished, but neither a regional differentiation nor a charging availability probability or distribution are assumed.

At the end of the execution of the GridModeler class, the available charging power during parking times is added to the activities dataframe.

In [7]:
# Run the first two classes to generate data

data = parse_data(configs=configs)
data= data.process()

Generic file parsing properties set up.
Starting to retrieve local data file from c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\tutorials\data_sampling\MiD17.csv.
Finished loading 2124 rows of raw data of type .csv.
Running in debug mode.
Finished harmonization of variables.
Finished harmonization of ID variables.
Starting filtering, applying 8 filters.
All filters combined yielded that a total of 241 trips are taken into account.
This corresponds to 40.166666666666664 percent of the original data.
Completed park timestamp adjustments.
Finished activity composition with 240 trips and 240 parking activites.


OSError: Cannot save file into a non-existent directory: 'c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\output\dataparser'

In [ ]:
grid = GridModeller(configs=configs, activities=data)
grid.assign_grid()

In [ ]:
# Estimate charging flexibility based on driving profiles and charge connection
flex = FlexEstimator(configs=configs, activities=grid.activities)
flex.estimate_technical_flexibility_through_iteration()

diary = DiaryBuilder(configs=configs, activities=flex.activities)
diary.create_diaries()


profiles = ProfileAggregator(configs=configs, activities=diary.activities, profiles=diary)
profiles.aggregate_profiles()


plt.figure(figsize=(10, 6))
plt.plot(profiles.uncontrolled_charging_weekly.index, profiles.uncontrolled_charging_weekly.iloc[:])  
plt.xlim(profiles.uncontrolled_charging_weekly.index[0], profiles.uncontrolled_charging_weekly.index[-1])
plt.title('Uncontrolled Charge Volume')
plt.xlabel('Time')
plt.ylabel('Charge Volume')
plt.grid(True)
plt.show()


We can now change the grid availability from home to the workplace only and see how this affects the available charging flexibility.

In [ ]:
configs['user_config']['gridmodellers']['charging_infrastructure_mappings']['HOME'] = False
configs['user_config']['gridmodellers']['charging_infrastructure_mappings']['WORK'] = True
configs['user_config']['gridmodellers']['charging_infrastructure_mappings']

In [ ]:
grid = GridModeller(configs=configs, activities=data)
grid.assign_grid()



In [ ]:
# Estimate charging flexibility based on driving profiles and charge connection
flex = FlexEstimator(configs=configs, activities=grid.activities)
flex.estimate_technical_flexibility_through_iteration()

diary = DiaryBuilder(configs=configs, activities=flex.activities)
diary.create_diaries()


profiles = ProfileAggregator(configs=configs, activities=diary.activities, profiles=diary)
profiles.aggregate_profiles()

plt.figure(figsize=(10, 6))
plt.plot(profiles.charging_power_weekly.index, profiles.charging_power_weekly.iloc[:])  
plt.xlim(profiles.charging_power_weekly.index[0], profiles.charging_power_weekly.index[-1])
plt.title('Uncontrolled Charge Volume')
plt.xlabel('Time')
plt.ylabel('Charge Volume')
plt.grid(True)

plt.show()

From the plots we can see how different the charging power looks, when the option of home charging is not available.  

## Next Steps

In the next tutorial, you will learn more in detail the internal workings of the FlexEstimator class and how to customise some settings.